In [ ]:
import pandas as pd
import os

# DATASET FROM: http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# GF TODO: check if excel present, otherwise download it
# wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

excel_with_path  = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
csv_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.csv'

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster using it than opening excel files\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)

In [ ]:
# define transaction: total cash for a single 'row' of the dataset, i.e. item_price times the number of items bought
df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# day, week and time are added for convenience of analysis later on
from datetime import datetime
df['InvoiceDay'] = df['InvoiceDate'].map(lambda p: p.date())
df['InvoiceWeekDay'] = df['InvoiceDate'].map(lambda p: p.weekday())
df['InvoiceTime'] = df['InvoiceDate'].map(lambda p: p.time())
df['InvoiceWeek'] = df['InvoiceDate'].map(lambda p: p.isocalendar()[1]+52*(p.year-2010))

# avoide negative weeks and start counting from the first week of the dataset, which starts from 01/12/2009
df['InvoiceWeek'] = df['InvoiceWeek']+3


# implement the definition of cancellation in the documentation:
# http://archive.ics.uci.edu/ml/datasets/Online+Retail+II
#      ==> "If this code starts with the letter 'c', it indicates a cancellation.""
def is_cancellation(s):
    if isinstance(s, int):
        #print "ordinary invoice"
        return 0
    elif isinstance(s, unicode):
        #print "unicode string"
        #thse are cancellations
        if s.rfind('C')!=-1:
            return 1
        else:
            # print "Something unexpected"  # found, e.g. A506401
            # print s
            return 2
df['IsCancellation'] = df['Invoice'].map(is_cancellation)     
      


# items with prices above ~1500 are so few that is worth looking at them in detail, and excluding them from the plots
# There'a lot of transactions with price set to 0, which based on the descriptions are 
max_item_price=1400
r             =(0,max_item_price)
def is_ordinary_item(p):
    if abs(p)>max_item_price or p==0:
        return 0
    else:
        return 1
df['IsOrdinaryItem'] = df['Price'].map(is_ordinary_item)        

# df.to_csv(csv_with_path, encoding = 'utf-8')

In [ ]:
# what are the nulls in each feature?
nulls = df.isnull().sum()[df.isnull().sum() != 0]

# select all transactions containing 
df_nulls = df[df.isnull().any(axis=1)].copy(deep=True)

nulls_rel = nulls/df.shape[0]*100

nulls_summary = pd.concat([nulls, nulls_rel], axis=1, keys=['nulls', 'rel. nulls [%]'])\
               .sort_values('nulls', ascending=False)

nulls_summary

#<a id='the_destination'></a>

## todo
- nan and empty -> cleanup   # IN THE MAKING -> only consequences on the users study
- add colums for data and time, separately ?    # DONE
- create a second DF by user: RFM added to


## What do I want to extract from this dataset?
- https://en.wikipedia.org/wiki/Exploratory_data_analysis

- how many customers   DONE, also as a function of country
- make a pliot of transaction, of purchase prices, and of invoice_price! #  DONE

- VS TIME: trends of spending overall, by country
- revenue as a function of date, as a function of the time in the day  # DONE
- trends of spending: overall, by the largest customer, by the smaller customers (TOO SPECIFIC?)

- NEED TO BUILD A PER CUSTOMER DF
- customer: how many transactions, how much total revenue:
    => BREAK DOWN BY country, tra
- RMF


- ==> do this
- how many different types of items                  # DONE
- what kind of items are bought the most (by NUMBER of by REVENUE), are cancelled the most, 
- how much revenue per type of item bought

- correlation between
- cancellations: fraction of cancellation by nunber of transactions and by proportion of renenue
-                correlation to CHURNING ? Correlation to country OR type of good purchased ?

- cust.groupby('customer_unique_id').size().value_counts() => The majority of customers made only a single purchase. # DONE


- Can I cathegorise the purchasable items? => If so, customers split across those cathegories
- Persona; how many cheap items, few expensive ones?
- https://cxl.com/blog/creating-customer-personas-using-data-driven-research/
- now many items are bought per session, how much is spent per session

==> TOWARDS CUSTOMER PERSONA DEFINITION
- MANY TRANSACTIONS ARE RECORDED AT THE SAME TIME => as if they were a single shopping session, but billed in split goups of goods
- the function summary_data_from_transaction_data treats transactions taking place on the same day as A SINGLE ONE!

---
<a id="section_ID_GF"></a>

- this section can be reached at this URL https://swan001.cern.ch/user/franzoni/notebooks/SWAN_projects/proton/proton-EDA.ipynb#section_ID_GF
- thanks to this documentation

<a id="section_customers"></a>
## Customers carachteristics

## plan for customers
- make a dataframe
- add also the RFM variables
- add nuber of items bought in total, number of different TYPEs of items
- add total value of course
- Can I cathegorise the purchasable items? => If so, customers split across those cathegories
- Persona; how many cheap items, few expensive ones?
- https://cxl.com/blog/creating-customer-personas-using-data-driven-research/

- DF to containg:
- filter out non standard AND null customers SHALL I REMOVE CANCELLATIONS ????? => yes, keep it basic for now!   DONE
- RFM
- PER TOTAL:    number of items per customer, number of DIFFERENT/unique items per customer             DONE
- PER INVOICE:  same-as-above  - DEFERED
- What's the number of orders/invoices per customer? DONE
- number of items per purchase, amount per purchase - of each different customer (do I alraedy have this ?)

+ RFM ANALYSIS:
- ==> add to the dataframe the quantiles of RFM, with the correct ordering/SCORE
- https://joaocorreia.io/blog/rfm-analysis-increase-sales-by-segmenting-your-customers.html
- https://www.blastanalytics.com/blog/rfm-analysis-boosts-sales

+ CLUSTERING
- https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set#The_silhouette_method

<a id='the_destination'></a>

[Pippo goes to the cinema](section-title)

# Cleanup of data for customer analysis
- we decide to remove all datata points with no Customer_ID
- and to limit the study only to IsOrdinaryItem (as defined above), to avoid skewing the resuts w/ dataa points which are not actual purchases
- to keep the modelling basic, for now we decide also to remove all cancellation oders



In [ ]:
df.shape

In [ ]:
df=df.dropna()

In [ ]:
df.shape

In [ ]:
df= df.loc[ (df.IsCancellation==0) & (df.IsOrdinaryItem==1) ]    

In [ ]:
df.shape

In [ ]:
df=df.sort_values(by=['InvoiceDate'],ascending=True)

# Basic customer integral features
- number of items bought trgought the entire period covered by data, indicating the overall commercial activity of each individual
- unique items, indicates the variety of the customers interests
- total amount spent by a customer
- number of invoices, i.e. how many times a customers surfaced and made an order

In [ ]:
df_customer_items  = df.groupby('Customer_ID').StockCode.agg([len])     \
                         .rename(columns={"len": "items"})               \
                         .sort_values(by=['items'],ascending=False)

tit='Num items bought by customer'
r=(0,3000)
df_customer_items.plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
df_customer_uniq_items  = df.groupby('Customer_ID').StockCode.unique().agg([len])     \
                         .rename(columns={"len": "uniq_items"})               \
                         .sort_values(by=['uniq_items'],ascending=False)

tit='Num unique items bought by customer'
r=(0,800)
df_customer_uniq_items.plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
df_customer_invoices  = df.groupby('Customer_ID').Invoice.unique().agg([len])     \
                         .rename(columns={"len": "invoices"})               \
                         .sort_values(by=['invoices'],ascending=False)

tit='Num invoices per customer'
r=(0,200)
df_customer_invoices.plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
df_customer_spent = df.groupby('Customer_ID').Transaction.agg([sum])     \
                   .rename(columns={"sum": "total_spent"})               \
                   .sort_values(by=['total_spent'],ascending=False)

tit='total amount spent by a customer'
r=(0,100000)
df_customer_spent.plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
df_customer_spent.sort_values(by=['Customer_ID'],ascending=True).head()

In [ ]:
df.loc[ (df.Customer_ID==12346) &  (df.Quantity<100000)].Transaction.sum()

In [ ]:
df.loc[ (df.Customer_ID==12346)]

In [ ]:
df.loc[ (df.Customer_ID==12346)].InvoiceDay.unique().shape
# verification of 'fequency' being equal to 7 for 12346
# remiminder: the RFM variables exclude the first invoice

# Historic customer  features
- **Recency**: How recently a customer has made a purchase
- **Frequency**: How often a customer makes a purchase
- **Monetary Value**: Average expenditure for an invoice

- note: RFM variables exlcude the first invoice for all customers. Only returning invoices are considered

In [ ]:
data_star='2009-12-01'
calib_end='2011-05-31'
obser_end='2011-12-09'

from datetime import date
d_data_star=date( * map(lambda u:int(u), data_star.split('-') ) )
d_calib_end=date( * map(lambda u:int(u), calib_end.split('-') ) )
d_obser_end=date( * map(lambda u:int(u), obser_end.split('-') ) )

from lifetimes.utils import *

#    This transforms a DataFrame of transaction data of the form:
#        customer_id, datetime [, monetary_value]
#    to a DataFrame of the form:
#        customer_id, frequency, recency, T [, monetary_value]

df_customer_RFM = summary_data_from_transaction_data(df.sort_values(by=['InvoiceDate'],ascending=True)
                                                     ,'Customer_ID','InvoiceDate'
                                                     ,monetary_value_col='Transaction'
                                                     ,freq='D'
                                                     #,observation_period_end=obser_end
                                                    )

In [ ]:
df_customer_RFM.head()

In [ ]:
tit='(Monetary Value) Average expenditure for an invoice'
r=(0,4000)
df_customer_RFM['monetary_value'].plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
tit='Frequency (how many times a customer has come back)'
r=(0,200)
df_customer_RFM['frequency'].plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
df_customer = df_customer_items     \
                .join(df_customer_invoices)    \
                .join(df_customer_spent)   \
                .join(df_customer_RFM)   \
            .sort_values(by=['total_spent'],ascending=False)

df_customer.head()